In [1]:
%cd /home/q123/Desktop/explo
import torch
import math 
from src.optim import step

import logging
logging.basicConfig(filename='myapp.log', level=logging.CRITICAL)

/home/q123/Desktop/explo


/home/q123/miniconda3/envs/explo/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# use a GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.float

# use regular spaced points on the interval [0, 1]
train_X = torch.linspace(0.3, 0.5, 15, dtype=dtype, device=device)
# training data needs to be explicitly multi-dimensional
train_X = train_X.unsqueeze(1)

# sample observed values and add some synthetic noise
train_Y = torch.sin(train_X * (2 * math.pi)) + 0.15 * torch.randn_like(train_X)
train_Y = train_Y
objective_env = lambda x : torch.sin(x).squeeze(dim=1) +  0.15 * torch.rand(1)

In [6]:
from botorch.models import SingleTaskGP
from gpytorch.constraints import GreaterThan
from gpytorch.mlls import ExactMarginalLogLikelihood

model = SingleTaskGP(train_X=train_X, train_Y=train_Y)
model.likelihood.noise_covar.register_constraint("raw_noise", GreaterThan(1e-5))
mll = ExactMarginalLogLikelihood(likelihood=model.likelihood, model=model)
# set mll and all submodules to the specified dtype and device
mll = mll.to(train_X)

In [7]:
max_iter = 20
for i in range(max_iter):
    
  new_x,new_y = step(model,objective_env)
  #print("outputscale",model.covar_module.raw_outputscale)
  best_val = model.train_targets.max()
  curr_val = model.train_targets[-1]
  print(f'curr {curr_val} max {best_val}')
  print(f'new_x {new_x} new_y {new_y}')

curr 0.009023568592965603 max 0.8074154853820801
new_x tensor([[0.]]) new_y tensor([0.0090])
curr 0.8513906598091125 max 0.8513906598091125
new_x tensor([[1.]]) new_y tensor([0.8514])
curr 0.8538792729377747 max 0.8538792729377747
new_x tensor([[0.8971]]) new_y tensor([0.8539])
curr 0.9095646739006042 max 0.9095646739006042
new_x tensor([[0.9500]]) new_y tensor([0.9096])
curr 0.8682857155799866 max 0.9095646739006042
new_x tensor([[0.9501]]) new_y tensor([0.8683])
curr 0.8652675151824951 max 0.9095646739006042
new_x tensor([[1.]]) new_y tensor([0.8653])
curr 0.9071233868598938 max 0.9095646739006042
new_x tensor([[0.9435]]) new_y tensor([0.9071])
curr 0.952530562877655 max 0.952530562877655
new_x tensor([[0.9443]]) new_y tensor([0.9525])
curr 0.7325518131256104 max 0.952530562877655
new_x tensor([[0.8214]]) new_y tensor([0.7326])
curr 0.8793346285820007 max 0.952530562877655
new_x tensor([[0.9126]]) new_y tensor([0.8793])
curr 0.899834394454956 max 0.952530562877655
new_x tensor([[0.91